In [1]:
#Importamos escalas
import pickle
with open('../data/scalers/scaler_X_ind.pkl', 'rb') as f:
    scaler_X_ind = pickle.load(f)

In [1]:
import streamlit as st
from tensorflow.keras.models import load_model
import pickle
import datetime
import pandas as pd
import numpy as np


In [2]:
#Importo df
df_ind = pd.read_csv('../data/DF_modelos/df_industrial_total.csv')
df_res = pd.read_csv('../data/DF_modelos/df_residencial_total.csv')
df_serv = pd.read_csv('../data/DF_modelos/df_servicios_total.csv')

#Importamos modelo
modelInd = load_model('../models/modelo_industrial_def.h5')
modelRes = load_model('../models/modelo_residencial_def.h5')
modelSer = load_model('../models/modelo_servicios_def.h5')

#Importamos escalas
with open('../data/scalers/scaler_X_ind.pkl', 'rb') as f:
    scaler_X_ind = pickle.load(f)
with open('../data/scalers/scaler_y_ind.pkl', 'rb') as f:
    scaler_y_ind = pickle.load(f)
with open('../data/scalers/scaler_X_res.pkl', 'rb') as f:
    scaler_X_res = pickle.load(f)
with open('../data/scalers/scaler_y_res.pkl', 'rb') as f:
    scaler_y_res = pickle.load(f)
with open('../data/scalers/scaler_X_serv.pkl', 'rb') as f:
    scaler_X_serv = pickle.load(f)
with open('../data/scalers/scaler_y_serv.pkl', 'rb') as f:
    scaler_y_serv = pickle.load(f)

In [6]:
def create_sequences(X, time_steps):
    Xs, ys = [], []
    # Ajuste para asegurarse de que no excedamos el rango del conjunto de datos
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
    return np.array(Xs)

In [18]:
def predecir_industrial(datos_usuario_ind):
    # Predecir los valores de prueba
    datos_usuario_df_ind = pd.DataFrame([datos_usuario_ind])
    Ultima_fila=df_ind.iloc[-1]
    df_datos = pd.concat([Ultima_fila_df, datos_usuario_df_ind], ignore_index=True)
    Ultima_fila_df = pd.DataFrame([Ultima_fila])
    datos_usuario_scaled_ind= scaler_X_ind.transform(datos_usuario_df_ind)
    time_steps = 2  # Esto significa que la predicción se basará en los últimos 2 dias
    datos_seq, = create_sequences(datos_usuario_scaled_ind, time_steps)
    predInd_scaled = modelInd.predict(datos_usuario_scaled_ind)
    # Desescalar las predicciones y los valores reales para comparar
    predInd = scaler_y_ind.inverse_transform(predInd_scaled)
    return predInd

In [34]:
def predecir_industrial(datos_usuario_ind):
    # Predecir los valores de prueba
    datos_usuario_df_ind = pd.DataFrame([datos_usuario_ind])
    df_ind['t-1'] = df_ind['consumo'].shift(1)
    # Obtener la última fila del DataFrame
    Ultima_fila = df_ind.iloc[-1]

    # Convertir la serie en un DataFrame de una sola fila
    Ultima_fila_df = pd.DataFrame([Ultima_fila])  

    # Concatenar la última fila con los datos del usuario
    df_datos = pd.concat([Ultima_fila_df, datos_usuario_df_ind], ignore_index=True)

    # Filtrar solo las columnas necesarias
    columnas_necesarias = ['findesemana','festivos', 'COVID', 'tmed', 't-1']
    
    # Asegurarse de que solo se están usando las columnas necesarias en el orden correcto
    df_datos_filtrados = df_datos[columnas_necesarias]

    # Imprimir las columnas para verificar el orden
    print("Columnas a escalar:", df_datos_filtrados.columns.tolist())
    print("Orden esperado:", columnas_necesarias)

    # Obtener las características del escalador
    feature_names = scaler_X_ind.get_feature_names_out()
    print("Características utilizadas en el ajuste del escalador:", feature_names)

    # Verificar que las columnas filtradas coincidan con las que se usaron para ajustar el escalador
    if not df_datos_filtrados.columns.equals(pd.Index(feature_names)):
        raise ValueError("Las columnas a escalar no coinciden con las características del escalador.")

    # Escalar los datos
    datos_usuario_scaled_ind = scaler_X_ind.transform(df_datos_filtrados)

    # Preparar las secuencias para la predicción
    time_steps = 2  # Esto significa que la predicción se basará en los últimos 2 días
    datos_seq = create_sequences(datos_usuario_scaled_ind, time_steps)

    # Realizar la predicción
    predInd_scaled = modelInd.predict(datos_seq)

    # Desescalar la predicción
    predInd = scaler_y_ind.inverse_transform(predInd_scaled)

    return predInd


In [35]:
temperatura_final=21
findesemana=0
festivos=0
t_1=df_ind['consumo'].iloc[-1]

datos_usuario = {
    "findesemana": findesemana,
    "festivos": festivos,
    "COVID": 0,
    "tmed": temperatura_final,
    "t-1": t_1
}

In [36]:
prediction = predecir_industrial(datos_usuario)
print(f"El día 2 se prevé un consumo de {prediction[0][0]:.2f}")

Columnas a escalar: ['findesemana', 'festivos', 'COVID', 'tmed', 't-1']
Orden esperado: ['findesemana', 'festivos', 'COVID', 'tmed', 't-1']
Características utilizadas en el ajuste del escalador: ['findesemana' 'festivos' 'COVID' 'tmed' 't-1']


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


ValueError: math domain error

In [3]:
# Comprobar si el escalador tiene atributos relevantes
if hasattr(scaler_X_ind, 'feature_names_in_'):
    print("Características del escalador:")
    print(scaler_X_ind.feature_names_in_)

print("Valores mínimos de las características:")
print(scaler_X_ind.data_min_)
print("Valores máximos de las características:")
print(scaler_X_ind.data_max_)


Características del escalador:
['findesemana' 'festivos' 'COVID' 'tmed' 't-1']
Valores mínimos de las características:
[0.00000e+00 0.00000e+00 0.00000e+00 1.47200e+00 4.44614e+05]
Valores máximos de las características:
[1.00000000e+00 1.00000000e+00 2.00000000e+00 3.12045455e+01
 3.58297500e+06]


In [4]:
# Comprobar si el escalador tiene atributos relevantes
if hasattr(scaler_X_res, 'feature_names_in_'):
    print("Características del escalador:")
    print(scaler_X_res.feature_names_in_)

print("Valores mínimos de las características:")
print(scaler_X_res.data_min_)
print("Valores máximos de las características:")
print(scaler_X_res.data_max_)

Características del escalador:
['findesemana' 'tmed' 'PIB' 't-1']
Valores mínimos de las características:
[0.00000e+00 1.47200e+00 4.93010e+04 6.11981e+05]
Valores máximos de las características:
[1.00000000e+00 3.12045455e+01 7.72680000e+04 1.33902900e+07]


In [5]:
# Comprobar si el escalador tiene atributos relevantes
if hasattr(scaler_X_serv, 'feature_names_in_'):
    print("Características del escalador:")
    print(scaler_X_serv.feature_names_in_)

print("Valores mínimos de las características:")
print(scaler_X_serv.data_min_)
print("Valores máximos de las características:")
print(scaler_X_serv.data_max_)

Características del escalador:
['findesemana' 'festivos' 'pernoctaciones' 'tmed' 'poblacion' 't-1']
Valores mínimos de las características:
[0.000000e+00 0.000000e+00 5.100000e+04 1.472000e+00 5.628011e+06
 4.942218e+06]
Valores máximos de las características:
[1.00000000e+00 1.00000000e+00 1.27000000e+06 3.12045455e+01
 5.88487300e+06 1.99296680e+07]
